<a href="https://colab.research.google.com/github/diggs1711/comma-ai-speed-challenge/blob/master/speed_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
from cv2 import VideoCapture, namedWindow, WINDOW_AUTOSIZE, waitKey
from google.colab.patches import cv2_imshow
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from pathlib import Path
drive_path = Path('/content/drive/My Drive/comma_ai_speed_challenge/speedchallenge-master/data/')

In [0]:
train_vals = np.array(pd.read_fwf(drive_path/'train.txt', header=None))

In [39]:
print(f"mean {train_vals.mean()}")
print(f"max {train_vals.max()}")
print(f"std {train_vals.std()}")
print(f"min {train_vals.min()}")

idx = int(len(train_vals) / 2)
print()
print("*** First half of video ****")
first_half = train_vals[:idx]
print(f"mean 1st {first_half.mean()}")
print(f"max 1st {first_half.max()}")
print(f"std 1st {first_half.std()}")
print(f"min 1st {first_half.min()}")

print()
print("*** Second half of video ****")
second_half = train_vals[idx:]
print(f"mean 2nd {second_half.mean()}")
print(f"max 2nd {second_half.max()}")
print(f"std 2nd {second_half.std()}")
print(f"min 2nd {second_half.min()}")

mean 12.183181660441177
max 28.130404
std 8.206561706745727
min 0.0

*** First half of video ****
mean 1st 18.66717958362745
max 1st 28.130404
std 1st 5.883530076403356
min 1st 0.0

*** Second half of video ****
mean 2nd 5.699183737254902
max 2nd 14.890168
std 2nd 3.9993656743963695
min 2nd 0.0


In [0]:
numberOfFrame = 20400
fps = 20
cropped_image_height = 100
cropped_image_width = 440

In [6]:
df.shape

(20400, 44000)

In [41]:
cap = VideoCapture('/content/drive/My Drive/comma_ai_speed_challenge/speedchallenge-master/data/train.mp4')
# driving containing 20400 frames. Video is shot at 20 fps.

i = 0
video_frames = np.array([])

while i < numberOfFrame:
  ret_val, frame = cap.read()
  print(frame.shape)
  break
  cropped_image = frame[200:200+cropped_image_height,100:100+cropped_image_width, 2]
  video_frames.append(np.array(cropped_image))
  i = i + 1
  if waitKey(1) == 27:
    break;

(480, 640, 3)


In [0]:
import math
valid_idx =  math.floor(numberOfFrame * 0.8)

X_train = video_frames[:valid_idx][..., np.newaxis]
y_train = train_vals[:valid_idx]

X_valid = video_frames[valid_idx:][..., np.newaxis]
y_valid = train_vals[valid_idx:]

In [26]:
X_train.shape

(16320, 100, 440, 1)

In [0]:
X_train = X_train / 255
X_valid = X_valid / 255

In [31]:
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D
from keras.models import Sequential
import keras

model=Sequential()
model.add(Conv2D(32, (3, 3),
                 strides=2,
                 padding='same',
                 input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, 3,3))
model.add(Flatten())
model.add(Dense(16))
model.add(Dense(1))

model.compile(loss='mse', optimizer=keras.optimizers.Adam(), metrics=['mse'])

model.fit(
    X_train,
    y_train,
    batch_size=16,
    epochs=100, 
    validation_data=(X_valid, y_valid),
    shuffle=False)

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  if sys.path[0] == '':


Train on 16320 samples, validate on 4080 samples
Epoch 1/100
16320/16320 [==============================] - 18s 1ms/step - loss: 13.2238 - mse: 13.2238 - val_loss: 46.8985 - val_mse: 46.8985
Epoch 2/100
16320/16320 [==============================] - 12s 727us/step - loss: 5.0269 - mse: 5.0269 - val_loss: 39.8841 - val_mse: 39.8841
Epoch 3/100
16320/16320 [==============================] - 12s 730us/step - loss: 4.8164 - mse: 4.8163 - val_loss: 53.4492 - val_mse: 53.4492
Epoch 4/100
16320/16320 [==============================] - 12s 734us/step - loss: 4.3588 - mse: 4.3588 - val_loss: 54.2479 - val_mse: 54.2479
Epoch 5/100
16320/16320 [==============================] - 12s 729us/step - loss: 4.5917 - mse: 4.5917 - val_loss: 13.6476 - val_mse: 13.6476
Epoch 6/100
16320/16320 [==============================] - 12s 728us/step - loss: 6.2395 - mse: 6.2395 - val_loss: 16.7369 - val_mse: 16.7369
Epoch 7/100
16320/16320 [==============================] - 12s 725us/step - loss: 6.0282 - mse: 6.0

KeyboardInterrupt: ignored